In [1]:
import pickle
from pedalboard import Pedalboard, Reverb, load_plugin
from pedalboard.io import AudioFile
from mido import Message
from IPython.display import Audio
import json
import os
import pathlib
import matplotlib.pyplot as plt

# Load the plugin
vital = load_plugin("/home/benjamin/Documents/work/Synthetizers-parameters-estimation/plugins/Vital.vst3")

# This will pause the script until you close the VST window
print("Opening plugin editor... Close the editor window to continue.")
vital.show_editor()

print("Reading parameters after manual edits...")

Opening plugin editor... Close the editor window to continue.
Reading parameters after manual edits...


In [2]:
print(vital.preset_data)

b'VST3\x01\x00\x00\x0056535456697461766974616C00000000\xbfs\x05\x00\x00\x00\x00\x00VstW\x00\x00\x00\x08\x00\x00\x00\x01\x00\x00\x00\x00CcnK\x00\x05swFBCh\x00\x00\x00\x02Vita\x00\x01\x05\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05r\xdf{"author":"Bold Void","comments":"www.boldvoid.com","macro1":"MACRO 1","macro2":"MACRO 2","macro3":"MACRO 3","macro4":"MACRO 4","preset_name":"piano","preset_style":"Keys","settings":{"beats_per_minute":2.0,"bypass":0.0,"chorus_cutoff":60.0,"chorus_delay_1":-9.0,"chorus

In [3]:
# Save as a vstpreset. Note that vital.preset_data is already in the correct binary format.
filename = "vital.vstpreset"
with open(filename, "wb") as f:
    f.write(vital.preset_data)
print(f"Saved preset to {pathlib.Path(filename).absolute()}")

Saved preset to /home/benjamin/Documents/work/Synthetizers-parameters-estimation/scripts/presets_dl/vital.vstpreset


In [4]:
# Play with the plugin parameters
sample_rate = 44100
sound = vital([Message("note_on", note=72), Message("note_off", note=72, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound, rate=sample_rate)

In [5]:
sound2 = vital([Message("note_on", note=50), Message("note_off", note=50, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound2, rate=sample_rate)

In [6]:
import json

preset_file = '/home/benjamin/Documents/work/Synthetizers-parameters-estimation/scripts/presets_dl/float.vital'
with open(preset_file, 'rb') as f:
    preset_data = json.load(f)

CORE_COMPONENT_MAP = {
    # Component Mapping (Prefixes)
    "osc": "oscillator",
    "env": "envelope",
    "random": "random_lfo",
    
    # Macro Control Mapping
    "macro_control": "macro",
    
    # Effect Mapping
    "reverb": "reverb",
    "chorus": "chorus",
    "phaser": "phaser",
    "flanger": "flanger",
    "eq": "eq",
    "delay": "delay",
    "compressor": "compressor", # Stays 'compressor' in both, though Set 2 uses specific band terms
    
    # Other Core Modules
    "sample": "sample",
    "wavetables": "wavetables",
    "lfos": "lfos",
    "modulations": "modulations"
}

COMMON_SUFFIX_MAP = {
    # On/Off
    "_on": "_switch",
    
    # Envelope Parameter Mapping
    "_attack": "_attack",
    "_decay": "_decay",
    "_sustain": "_sustain",
    "_release": "_release",
    "_hold": "_hold",
    
    # LFO Keytracking
    "_keytrack_tune": "_tune",
    "_keytrack_transpose": "_transpose",
    "_fade_time": "_fade_in",
    
    # Oscillator Specific
    "_spectral_morph": "_frequency_morph",
    "_random_phase": "_phase_randomization",
    "_frame_spread": "_frame_spread", # Note: Set 2 often uses "unison_frame_spread" for this context
    
    # Effect Mix
    "_dry_wet": "_mix",
    
    # Filter/Effect Keytracking
    "_keytrack": "_key_track",
}

preset_new_settings = {}
for key, value in preset_data["settings"].items():
    new_key = key
    for prefix, mapped_prefix in CORE_COMPONENT_MAP.items():
        if key.startswith(prefix):
            new_key = key.replace(prefix, mapped_prefix, 1)
            break
    for suffix, mapped_suffix in COMMON_SUFFIX_MAP.items():
        if new_key.endswith(suffix):
            new_key = new_key[:-len(suffix)] + mapped_suffix
            break
    preset_new_settings[new_key] = value

print(f"Length of vital parameters: {len(vital.parameters)}")
print(f"Length of preset settings before remapping: {len(preset_data['settings'])}")

# Print common length after remapping
common_keys_remapped = set(preset_new_settings.keys()).intersection(set(vital.parameters.keys()))
print(len(common_keys_remapped))

# Show keys in preset but not in vital parameters after remapping
preset_only_keys_remapped = set(preset_new_settings.keys()) - set(vital.parameters.keys())
print(preset_only_keys_remapped)

# Show keys in vital parameters but not in preset after remapping
vital_only_keys_remapped = set(vital.parameters.keys()) - set(preset_new_settings.keys())
print(vital_only_keys_remapped)

Length of vital parameters: 775
Length of preset settings before remapping: 775
722
{'oscillator_3_spectral_morph_spread', 'lfo_8_delay_time', 'lfos', 'filter_2_blend_transpose', 'sample_phase_randomization', 'compressor_high_upper_ratio', 'oscillator_2_spectral_morph_amount', 'filter_fx_blend_transpose', 'wavetables', 'lfo_7_delay_time', 'compressor_high_lower_ratio', 'oscillator_1_spectral_morph_spread', 'delay_aux_tempo', 'oscillator_3_unison_blend', 'lfo_3_delay_time', 'lfo_5_delay_time', 'oscillator_2_spectral_morph_spread', 'compressor_low_upper_threshold', 'compressor_high_lower_threshold', 'sample_key_track', 'chorus_cutoff', 'oscillator_2_frame_spread', 'lfo_1_delay_time', 'delay_aux_sync', 'oscillator_3_frame_spread', 'oscillator_1_unison_blend', 'compressor_band_lower_ratio', 'compressor_low_lower_ratio', 'oscillator_2_spectral_morph_type', 'delay_aux_frequency', 'compressor_low_lower_threshold', 'lfo_2_delay_time', 'lfo_4_delay_time', 'compressor_band_upper_ratio', 'oscilla

In [7]:
# Reload the plugin
vital_reset = load_plugin("/home/benjamin/Documents/work/Synthetizers-parameters-estimation/plugins/Vital.vst3")

# Load the parameters
vital_reset.load_preset(filename)

In [8]:
# Play with the reloaded plugin parameters
sample_rate = 44100
sound = vital_reset([Message("note_on", note=72), Message("note_off", note=72, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound, rate=sample_rate)

In [9]:
sound2 = vital_reset([Message("note_on", note=50), Message("note_off", note=50, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound2, rate=sample_rate)

In [10]:
vital_settings = load_plugin("/home/benjamin/Documents/work/Synthetizers-parameters-estimation/plugins/Vital.vst3")
vital_settings.show_editor()
# load settings from preset_new_settings
success = 0
for key, value in preset_new_settings.items():
    if key in vital_settings.parameters:
        try:
            # Cast to bool, string, int, or float as needed
            param_type = type(vital_settings.parameters[key])
            if param_type == bool:
                value = bool(value)
            elif param_type == int:
                value = int(value)
            elif param_type == float:
                value = float(value)
            elif param_type == str:
                value = str(value)
            setattr(vital_settings, key, value)
            success += 1
        except Exception as e:
            continue
print("Done loading parameters manually.")
print(f"Successfully set {success} parameters out of {len(preset_new_settings)}")

# Saved parameters as vstpreset
filename = "/home/benjamin/Documents/work/Synthetizers-parameters-estimation/scripts/presets_dl/vital2.vstpreset"
with open(filename, "wb") as f:
    f.write(vital_settings.preset_data)
print(f"Saved preset to {pathlib.Path(filename).absolute()}")

Done loading parameters manually.
Successfully set 318 parameters out of 775
Saved preset to /home/benjamin/Documents/work/Synthetizers-parameters-estimation/scripts/presets_dl/vital2.vstpreset


In [11]:
vital_settings = load_plugin("/home/benjamin/Documents/work/Synthetizers-parameters-estimation/plugins/Vital.vst3")
vital_settings.load_preset(filename)

In [12]:
# Play with the reloaded plugin parameters
sample_rate = 44100
sound = vital_settings([Message("note_on", note=72), Message("note_off", note=72, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound, rate=sample_rate)

In [13]:
sound2 = vital_settings([Message("note_on", note=50), Message("note_off", note=50, time=2)],
        sample_rate=sample_rate,
        duration=2,
        num_channels=2
)
Audio(sound2, rate=sample_rate)